In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-spam-detection-dataset-classification/spam.csv


In [2]:
# Imports and customizations
import os
import glob
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import nltk
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try: 
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    nltk.download('omw-1.4')
import gensim.downloader as api
import xgboost as xgb
import string


from pathlib import Path
from tqdm.auto import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from scipy.sparse import hstack

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [3]:
# Finding and downloading a dataset (working on Kaggle)
input_path = '/kaggle/input'

if os.path.exists(input_path):
    print(f"Папка {input_path} існує.")
    for dirname, _, filenames in os.walk(input_path):
        print(f"📂 Папка: {dirname}")
        for filename in filenames:
            print(f"  - {filename}")
else:
    print(f"Папка {input_path} не знайдена. Перевір підключення датасету.")


Папка /kaggle/input існує.
📂 Папка: /kaggle/input
📂 Папка: /kaggle/input/email-spam-detection-dataset-classification
  - spam.csv


In [4]:
data_path = '/kaggle/input/email-spam-detection-dataset-classification/spam.csv'
df = pd.read_csv(data_path, encoding='latin-1')

print('Review first 5 rows of dataset:')
display(df.head(10))

Review first 5 rows of dataset:


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [5]:
print('\n Information about dataset:')
df.info()


 Information about dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
print('\Checking for missing values:')
print(df.isnull().sum())

\Checking for missing values:
v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64


In [7]:
# choose only usefull columns and renamed their
df = df[['v1', 'v2']].rename(columns={'v1':'label','v2':'text'})

In [8]:
print('\nUnique values in column "Label":')
print(df['label'].unique())


Unique values in column "Label":
['ham' 'spam']


In [9]:
print('\nMissing in the text:', df['text'].isnull().sum())

display(df.sample(5, random_state = 37))


Missing in the text: 0


,label,text
895,ham,Superb Thought- \Be grateful that u dont have ...
4736,ham,Nt only for driving even for many reasons she ...
5512,ham,"Just making dinner, you ?"
4553,ham,Try to do something dear. You read something f...
3415,ham,Uhhhhrmm isnt having tb test bad when youre sick


In [10]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Lowercase
    text = text.lower()
    #2. Remove everything that is not letters and spaces
    text = re.sub(r'[^a-z\s]','', text)
    # 3. Break text into words
    words = text.split()
    # 4. Remove stop words and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    #5. Connect back into a string
    return ' '.join(words)

# check in exampes
print(clean_text('Superb Thought- \\Be grateful that u dont have ...'))

superb thought grateful u dont


In [11]:
df['clean_text'] = df['text'].apply(clean_text)

In [12]:
# Splitting into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['clean_text'], df['label'], test_size=0.2, random_state=37, stratify = df['label'])

print(f'Size of train set:{len(X_train)}')
print(f'Size of validation set:{len(X_val)}')

Size of train set:4457
Size of validation set:1115


In [13]:
# Text vectorization (BoW and TF-IDF)
# Initialize vectorizers
count_vectorizer = CountVectorizer(max_features=5000, ngram_range=(1,2))
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

# We train and transform training data
X_train_count = count_vectorizer.fit_transform(X_train)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform validation data
X_val_count = count_vectorizer.transform(X_val)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

print(f'Dimensionality of the BoW matrix: {X_train_count.shape}')
print(f'Dimensionality of TF-IDF matrix:{X_train_tfidf.shape}')

Dimensionality of the BoW matrix: (4457, 5000)
Dimensionality of TF-IDF matrix:(4457, 5000)


In [14]:
#Training Logistic Regression models on BoW and TF-IDF
# Initialize models
lr_count = LogisticRegression(max_iter = 1000, random_state = 37)
lr_tfidf = LogisticRegression(max_iter = 1000, random_state = 37)

# Training on BoW
lr_count.fit(X_train_count, y_train)

# Training on TF-IDF
lr_tfidf.fit(X_train_tfidf, y_train)

# Predictions

y_pred_count = lr_count.predict(X_val_count)
y_pred_tfidf = lr_tfidf.predict(X_val_tfidf)

# Spam class probability predictions
y_proba_count = lr_count.predict_proba(X_val_count)[:, 1]
y_proba_tfidf = lr_tfidf.predict_proba(X_val_tfidf)[:, 1]

# Rating of model

print("=== Logistic Regression on BoW ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_count):.4f}")
print(f"AUC: {roc_auc_score(y_val.map({'ham':0, 'spam':1}), y_proba_count):.4f}")
print(classification_report(y_val, y_pred_count))

print("\n=== Logistic Regression on TF-IDF ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_tfidf):.4f}")
print(f"AUC: {roc_auc_score(y_val.map({'ham':0, 'spam':1}), y_proba_tfidf):.4f}")
print(classification_report(y_val, y_pred_tfidf))

=== Logistic Regression on BoW ===
Accuracy: 0.9803
AUC: 0.9880
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.85      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115


=== Logistic Regression on TF-IDF ===
Accuracy: 0.9605
AUC: 0.9883
              precision    recall  f1-score   support

         ham       0.96      0.99      0.98       966
        spam       0.96      0.74      0.83       149

    accuracy                           0.96      1115
   macro avg       0.96      0.87      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [15]:
# Using pre-trained embeddings (Word2Vec)
# Download model Word2Vec
w2v_model = api.load("glove-wiki-gigaword-50")
print("GloVe model downloaded")


[==================================================] 100.0% 66.0/66.0MB downloaded
GloVe model downloaded


In [16]:
# Function to get embedding vectors for text
def get_embedding_vector(text, model, vector_size=50):
    words = text.split()
    valid_words = [word for word in words if word in model.key_to_index]
    if not valid_words:
        return np.zeros(vector_size)
    word_vectors = [model[word] for word in valid_words]
    # Make sure that all vectors are the same size
    for vec in word_vectors:
        assert len(vec) == vector_size
    return np.mean(word_vectors, axis=0)

In [17]:
# We obtain embedding vectors for the training and validation sets
X_train_w2v = np.array([get_embedding_vector(text, w2v_model) for text in X_train])
X_val_w2v = np.array([get_embedding_vector(text, w2v_model) for text in X_val])

In [18]:
# Training a model on embedding vectors (Logistic Regression)
lr_w2v = LogisticRegression(max_iter = 1000, random_state = 37)
lr_w2v.fit(X_train_w2v, y_train)

y_pred_w2v = lr_w2v.predict(X_val_w2v)
y_proba_w2v = lr_w2v.predict_proba(X_val_w2v)[:,1]

print("=== Logistic Regression on Word2Vec embedding ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_w2v):.4f}")
print(f"AUC: {roc_auc_score(y_val.map({'ham':0, 'spam':1}), y_proba_w2v):.4f}")
print(classification_report(y_val, y_pred_w2v))

=== Logistic Regression on Word2Vec embedding ===
Accuracy: 0.8951
AUC: 0.9257
              precision    recall  f1-score   support

         ham       0.93      0.95      0.94       966
        spam       0.62      0.56      0.59       149

    accuracy                           0.90      1115
   macro avg       0.78      0.75      0.76      1115
weighted avg       0.89      0.90      0.89      1115



In [19]:
#  Random Forest on BoW
rf_count = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_count.fit(X_train_count, y_train)

y_pred_rf_count = rf_count.predict(X_val_count)
y_proba_rf_count = rf_count.predict_proba(X_val_count)[:, 1]

print("=== Random Forest on BoW ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf_count):.4f}")
print(f"AUC: {roc_auc_score(y_val.map({'ham':0, 'spam':1}), y_proba_rf_count):.4f}")
print(classification_report(y_val, y_pred_rf_count))

=== Random Forest on BoW ===
Accuracy: 0.9767
AUC: 0.9859
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       966
        spam       0.99      0.83      0.91       149

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [20]:
# Random Forest on TF-IDF

rf_tfidf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_tfidf.fit(X_train_tfidf, y_train)

y_pred_rf_tfidf = rf_tfidf.predict(X_val_tfidf)
y_proba_rf_tfidf = rf_tfidf.predict_proba(X_val_tfidf)[:, 1]

print("=== Random Forest on TF-IDF ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf_tfidf):.4f}")
print(f"AUC: {roc_auc_score(y_val.map({'ham':0, 'spam':1}), y_proba_rf_tfidf):.4f}")
print(classification_report(y_val, y_pred_rf_tfidf))


=== Random Forest on TF-IDF ===
Accuracy: 0.9767
AUC: 0.9893
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99       966
        spam       1.00      0.83      0.90       149

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [21]:
# XGBoost on TF-IDF

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)  # ham -> 0, spam -> 1
y_val_enc = le.transform(y_val)


xgb_tfidf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_tfidf.fit(X_train_tfidf, y_train_enc)

y_pred_xgb_tfidf = xgb_tfidf.predict(X_val_tfidf)
y_proba_xgb_tfidf = xgb_tfidf.predict_proba(X_val_tfidf)[:, 1]

print("=== XGBoost on TF-IDF ===")
print(f"Accuracy: {accuracy_score(y_val_enc, y_pred_xgb_tfidf):.4f}")
print(f"AUC: {roc_auc_score(y_val_enc, y_proba_xgb_tfidf):.4f}")
print(classification_report(y_val_enc, y_pred_xgb_tfidf, target_names=le.classes_))

=== XGBoost on TF-IDF ===
Accuracy: 0.9713
AUC: 0.9754
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       966
        spam       0.98      0.81      0.88       149

    accuracy                           0.97      1115
   macro avg       0.97      0.90      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [22]:
# SVM on TF-IDF

svm_tfidf = SVC(kernel='linear', probability=True, random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train)

y_pred_svm_tfidf = svm_tfidf.predict(X_val_tfidf)
y_proba_svm_tfidf = svm_tfidf.predict_proba(X_val_tfidf)[:, 1]

print("=== SVM on TF-IDF ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_svm_tfidf):.4f}")
print(f"AUC: {roc_auc_score(y_val.map({'ham':0, 'spam':1}), y_proba_svm_tfidf):.4f}")
print(classification_report(y_val, y_pred_svm_tfidf))

=== SVM on TF-IDF ===
Accuracy: 0.9830
AUC: 0.9918
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       0.97      0.90      0.93       149

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



Порівняння моделей на TF-IDF:

| Модель        | Accuracy | AUC    | Precision (spam) | Recall (spam) | F1 (spam) |
|---------------|----------|--------|------------------|---------------|-----------|
| LogisticReg   | 0.9614   | 0.9880 | 0.96             | 0.74          | 0.84      |
| Random Forest | 0.9767   | 0.9893 | 1.00             | 0.83          | 0.90      |
| SVM           | 0.9830   | 0.9918 | 0.97             | 0.90          | 0.93      |
| XGBoost       | 0.9713   | 0.9754 | 0.98             | 0.81          | 0.88      |


**Висновок:**
 * SVM — найкраща модель по балансу точності та повноти (precision та recall).

 * Random Forest і XGBoost теж сильні, але трохи поступаються SVM.

 * Logistic Regression — базова модель, теж робить гарну роботу, але слабша за ансамблеві методи.

In [23]:
# Hyperparameter search for SVM (GridSearchCV)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']  # for 'rbf' kernel
}

svm = SVC(probability=True, random_state=37)

grid_search = GridSearchCV(svm, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=2)
grid_search.fit(X_train_tfidf, y_train_enc)

print("The Best parameters:", grid_search.best_params_)
print("The best AUC:", grid_search.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
The Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
The best AUC: 0.9861314378808341


In [24]:
# Training a model with the best parameters
best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_val_tfidf)
y_proba_best = best_svm.predict_proba(X_val_tfidf)[:, 1]

print("=== The Best SVM on TF-IDF ===")
print(f"Accuracy: {accuracy_score(y_val_enc, y_pred_best):.4f}")
print(f"AUC: {roc_auc_score(y_val_enc, y_proba_best):.4f}")
print(classification_report(y_val_enc, y_pred_best, target_names=le.classes_))

=== The Best SVM on TF-IDF ===
Accuracy: 0.9803
AUC: 0.9902
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       0.98      0.87      0.92       149

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [25]:
#Let's add engineering features to the data
def add_features(df):
    df = df.copy()
    df['text_len'] = df['text'].apply(len)
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    df['digit_count'] = df['text'].apply(lambda x: sum(c.isdigit() for c in x))
    df['punctuation_count'] = df['text'].apply(lambda x: sum(c in string.punctuation for c in x))
    return df

# add engineering features to train and validate datasets
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
val_df = pd.DataFrame({'text': X_val, 'label': y_val})

train_df = add_features(train_df)
val_df = add_features(val_df)

In [26]:
# Let's combine TF-IDF vectors with new features
X_train_features = hstack([X_train_tfidf, train_df[['text_len', 'word_count', 'digit_count', 'punctuation_count']].values])
X_val_features = hstack([X_val_tfidf, val_df[['text_len', 'word_count', 'digit_count', 'punctuation_count']].values])

In [27]:
#Let's train SVM with new features (with better hyperparameters)
best_svm.fit(X_train_features, y_train_enc)

y_pred_new = best_svm.predict(X_val_features)
y_proba_new = best_svm.predict_proba(X_val_features)[:, 1]

print("=== SVM on TF-IDF + engeneer features ===")
print(f"Accuracy: {accuracy_score(y_val_enc, y_pred_new):.4f}")
print(f"AUC: {roc_auc_score(y_val_enc, y_proba_new):.4f}")
print(classification_report(y_val_enc, y_pred_new, target_names=le.classes_))


=== SVM on TF-IDF + engeneer features ===
Accuracy: 0.8789
AUC: 0.8661
              precision    recall  f1-score   support

         ham       0.90      0.97      0.93       966
        spam       0.59      0.30      0.40       149

    accuracy                           0.88      1115
   macro avg       0.75      0.63      0.67      1115
weighted avg       0.86      0.88      0.86      1115



In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Scaling engineering features
train_features_scaled = scaler.fit_transform(train_df[['text_len', 'word_count', 'digit_count', 'punctuation_count']])
val_features_scaled = scaler.transform(val_df[['text_len', 'word_count', 'digit_count', 'punctuation_count']])

# Combining TF-IDF and scaled features
X_train_features_scaled = hstack([X_train_tfidf, train_features_scaled])
X_val_features_scaled = hstack([X_val_tfidf, val_features_scaled])

# We train the model again
best_svm.fit(X_train_features_scaled, y_train_enc)

y_pred_scaled = best_svm.predict(X_val_features_scaled)
y_proba_scaled = best_svm.predict_proba(X_val_features_scaled)[:, 1]

print("=== SVM on TF-IDF + Scaling engineering features ===")
print(f"Accuracy: {accuracy_score(y_val_enc, y_pred_scaled):.4f}")
print(f"AUC: {roc_auc_score(y_val_enc, y_proba_scaled):.4f}")
print(classification_report(y_val_enc, y_pred_scaled, target_names=le.classes_))


=== SVM on TF-IDF + Scaling engineering features ===
Accuracy: 0.9830
AUC: 0.9925
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.87      0.93       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



**Висновок:**
<br>Прості інженерні ознаки можуть бути корисними, але для роботи з ними важливо правильно масштабувати дані, щоб алгоритм не «заплутався» через різні діапазони значень.

| Модель                      | Accuracy | AUC    | Precision (spam) | Recall (spam) | F1 (spam) |
|----------------------------|----------|--------|------------------|---------------|-----------|
| Logistic Regression (BoW)   | 0.9803   | 0.9881 | 1.00             | 0.85          | 0.92      |
| Logistic Regression (TF-IDF)| 0.9614   | 0.9880 | 0.96             | 0.74          | 0.84      |
| Random Forest (BoW)         | 0.9767   | 0.9859 | 0.99             | 0.83          | 0.91      |
| Random Forest (TF-IDF)      | 0.9767   | 0.9893 | 1.00             | 0.83          | 0.90      |
| SVM (TF-IDF)                | 0.9830   | 0.9918 | 0.97             | 0.90          | 0.93      |
| SVM (TF-IDF + scaled features)| **0.9830** | **0.9925** | **1.00**       | 0.87          | 0.93      |
| XGBoost (TF-IDF)            | 0.9713   | 0.9754 | 0.98             | 0.81          | 0.88      |
| Logistic Regression (Word2Vec)| 0.8951 | 0.9257 | 0.62             | 0.56          | 0.59      |


**Основні висновки:**
* Найкращі результати продемонструвала SVM-модель з TF-IDF векторизацією та масштабованими інженерними ознаками. Вона поєднує високу точність і збалансований показник recall для класу спаму, що є критично важливим для задачі виявлення небажаних повідомлень.

* TF-IDF показав кращі результати, ніж Bag-of-Words (BoW), особливо у поєднанні з SVM. Це пов’язано з тим, що TF-IDF враховує не тільки частоту, а й значущість слів у контексті всього корпусу.

* Попередньо навчені ембедінги Word2Vec не дали кращих результатів в порівнянні з класичними методами векторизації тексту. Це може бути пов’язано з обмеженнями розмірності, контексту та специфіки текстів у наборі даних.

* Інженерні ознаки без масштабування суттєво погіршували якість моделі, але правильне масштабування цих ознак дозволило покращити результат і підвищити AUC.

* Random Forest та XGBoost теж показали хороші результати, але все ж дещо поступаються оптимізованому SVM.

**Рекомендації:**
* Для завдань класифікації спаму варто використовувати TF-IDF векторизацію з оптимізованою SVM-моделлю.

* Варто продовжувати вдосконалювати інженерні ознаки, експериментувати з більш просунутими моделями (нейронні мережі, трансформери) і робити аналіз помилок для розуміння слабких місць системи.

* Для подальшого покращення можливе використання ансамблів моделей.